In [ ]:
from azureml.core import Workspace, Model, Run, Environment
import os

In [ ]:
ws = Workspace.from_config()

In [ ]:
model_list = Model.list(ws, 'MY_MODEL')
best_child_run_id = model_list[0].run_id
best_child_run = Run.get(ws, best_child_run_id)
model = model_list[0]
model

In [ ]:
#Deploy the model from the best run as an AKS web service
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice, LocalWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

environment = Environment.from_conda_specification('objectdetectionenv', './automl_outputs/outputs/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='./outputs/scoring_file_v_1_0_0.py', environment=environment, source_directory='./automl_outputs') 

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=deployment_config,
                           name='automl-image-test',
                           overwrite=True)
local_service.wait_for_deployment(show_output=True)
print(local_service.state)

In [ ]:
import requests

# URL for the web service
scoring_uri = local_service.scoring_uri

sample_image = './path/to/sample/img.jpg'

# Load image data
data = open(sample_image, 'rb').read()

# Set the content type
headers = {'Content-Type': 'application/octet-stream'}

In [ ]:
from IPython.display import Image 
img = Image(filename=sample_image)
display(img)

In [ ]:


%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
import numpy as np
import json

# Object detection threshold
minscore=0.8

for f in os.listdir('./FlyProject'):
    if 'jp' in f:
        pass
    else:
        continue
    sample_image = './FlyProject/{}'.format(f)
    data = open(sample_image, 'rb').read()
    # Set the content type
    headers = {'Content-Type': 'application/octet-stream'}

    # If authentication is enabled, set the authorization header
    # headers['Authorization'] = f'Bearer {key}'

    # Make the request and display the response
    resp = requests.post(scoring_uri, data, headers=headers)

    IMAGE_SIZE = (9,6)
    plt.figure(figsize=IMAGE_SIZE)
    img_np=mpimg.imread(sample_image)
    img = Image.fromarray(img_np.astype('uint8'),'RGB')
    x, y = img.size

    fig,ax = plt.subplots(1, figsize=(30,30))
    # Display the image
    ax.imshow(img_np)

    # draw box and label for each detection 
    detections = json.loads(resp.text)
    for detect in detections['boxes']:
        # print(detect)
        label = detect['label']
        box = detect['box']
        # polygon = detect['polygon']
        conf_score = detect['score']
        if conf_score >= minscore: #Min probability
                ymin, xmin, ymax, xmax =  box['topY'],box['topX'], box['bottomY'],box['bottomX']
                topleft_x, topleft_y = x * xmin, y * ymin
                width, height = x * (xmax - xmin), y * (ymax - ymin)
                # print('{}: [{}, {}, {}, {}], {}'.format(detect['label'], round(topleft_x, 3), 
                #                                         round(topleft_y, 3), round(width, 3), 
                #                                         round(height, 3), round(conf_score, 3)))

                color = 'red' #You can change the color
                rect = patches.Rectangle((topleft_x, topleft_y), width, height, 
                                         linewidth=3, edgecolor=color,facecolor='none')
                textdisp=label+'='+str(round(conf_score, 3))
                ax.add_patch(rect)
                plt.text(topleft_x, topleft_y - 10, textdisp, color=color, fontsize=18)

    print(f)
    plt.show()
    display(img)
    print()
    print()